# **M1 KIVA SML Model Loan Prediction**

## **Table of Contents**

* Libraries & Data Importing

* Data Preparation

* Filter Countries to Top 20

* Gender Mapping

* Calculating Funding Duration

* SML Preparation

* Exporting Files for Streamlit

## Model Description

#### Problem Statement:

The objective is to predict the loan amount that a borrower can receive based on various factors such as the borrower's country, sector, activity type, number of borrowers, funding duration, and repayment term. By understanding these relationships, the model can provide insights into how these factors influence the size of loans that borrowers are likely to be granted.

#### Type of Model:
The model uses an ensemble approach, combining both Random Forest and XGBoost models. This ensemble method takes advantage of the strengths of both algorithms, providing a robust and accurate prediction of loan amounts by reducing overfitting and improving model performance. The target variable, `loan_amount`, is continuous, making regression the appropriate method.

#### Objective:
This ensemble model aims to assist lending platforms and financial institutions in estimating the appropriate loan amount for borrowers under different conditions. The model's insights can help improve loan approval strategies, ensure fair and data-driven lending decisions, and effectively manage borrower expectations. By accurately predicting loan amounts, the model helps lenders optimize resources, minimize risk, and better serve their customers.
""")

### Libraries & Data importing

In [100]:
# install all requried libraries from requirement file (-r flag) and running in silent mode using -q flag
!pip install -q -r https://raw.githubusercontent.com/JAdamHub/M1-Exam-Submission/refs/heads/main/requirements.txt

In [101]:
#import libraries
import pandas as pd             # for data manipulation
import numpy as np              # for mathematical operations
import matplotlib.pyplot as plt # for plotting
from scipy.stats import zscore  # for standardizing data & removing outliers
from vega_datasets import data  # for data visualization

#additional libraries for importing data from github
import requests                 # used for importing data
import io                       # used for converting response content to a file-like object for pandas

# library used for gender mapping - section 4
import re

In [102]:
# definition of url-paths for datasets
# online
url1 = 'https://raw.githubusercontent.com/JAdamHub/M1-Exam-Submission/refs/heads/main/data/kiva_loans_part_0.csv'
url2 = 'https://raw.githubusercontent.com/JAdamHub/M1-Exam-Submission/refs/heads/main/data/kiva_loans_part_1.csv'
url3 = 'https://raw.githubusercontent.com/JAdamHub/M1-Exam-Submission/refs/heads/main/data/kiva_loans_part_2.csv'

# loading the urls into requests to download data
response1 = requests.get(url1)
response2 = requests.get(url2)
response3 = requests.get(url3)

# convert the response content to a file-like object for pandas using io.StringIO
# the advantage of StringIO is that it allows us to import the data without having to save it as a file for pandas
data_part1 = pd.read_csv(io.StringIO(response1.text))
data_part2 = pd.read_csv(io.StringIO(response2.text))
data_part3 = pd.read_csv(io.StringIO(response3.text))

# We can see, that the imported loan dataset consists of 3 parts. We will like to combine these to one dataset
data = pd.concat([data_part1, data_part2, data_part3])

### Data Preparation

In [103]:
# Dropping unnecessary columns
data = data.drop(['tags', 'use', 'currency', 'country_code', 'partner_id'], axis=1)

# we want to remove the missing rows from the dataset
# storing length of rows for comparing, >> before dropna.. <<
data_rows = len(data)

# dropping missing values
data.dropna(inplace=True)

# storing the now cleaned dataset
cleaned_rows = len(data)

# check..
drops = data_rows - cleaned_rows

# print the dropped rows and percentage of data removed
print(f"Number of dropped rows: {drops}")
print(f'In percentage {(drops / data_rows) * 100:.2f} % of the data was removed')
# check after cleaning
data.isna().sum()

Number of dropped rows: 97078
In percentage 14.46 % of the data was removed


id                    0
funded_amount         0
loan_amount           0
activity              0
sector                0
country               0
region                0
posted_time           0
disbursed_time        0
funded_time           0
term_in_months        0
lender_count          0
borrower_genders      0
repayment_interval    0
date                  0
dtype: int64

### Filter countries to only include top 20 based on total loan_amount

In [104]:
# what is the total amount of loan_amount for each country?
country_loans = data.groupby('country')['loan_amount'].sum()

# we would like the top 20 countries
country_top_20 = country_loans.sort_values(ascending=False).head(20)

# using index we create a new variable with the top 20 countries
data = data[data['country'].isin(country_top_20.index)]

### Gender Mapping

In [105]:
### Gender Mapping (creating groups of male, female, and mixed gender groups)
# count the number of male and female borrowers by searching for occurrences of 'male' and 'female' in the 'borrower_genders' column
data['male_borrowers'] = data['borrower_genders'].apply(lambda x: len(re.findall(r'\bmale', x)))
data['female_borrowers'] = data['borrower_genders'].apply(lambda x: len(re.findall(r'\bfemale', x)))

# calculate the total number of borrowers by adding male and female borrower counts
data['borrowers_count'] = data['male_borrowers'] + data['female_borrowers']

# handle cases where 'borrowers_count' is 0 by replacing it with 1 to avoid division by zero
data['male_borrower_ratio'] = data['male_borrowers'] / data['borrowers_count'].replace(0, 1)

# function to classify the gender group based on the ratio of male borrowers
# if the ratio is 1, it means all borrowers are male, so return 'male(s)'
# if the ratio is 0, all borrowers are female, so return 'female(s)'
# otherwise, return 'mixed gender group' for groups with both male and female borrowers
def classify_genders(ratio):
    if ratio == 1:
        return 'male(s)'
    elif ratio == 0:
        return 'female(s)'
    else:
        return 'mixed gender group'

# apply the gender classification function to each row based on the 'male_borrower_ratio'
data['gender_class'] = data['male_borrower_ratio'].apply(classify_genders)

# print of the first 10 rows of new added columns to see the borrower genders, counts, ratios, and classifications
data[['borrower_genders', 'male_borrowers', 'female_borrowers', 'borrowers_count', 'male_borrower_ratio', 'gender_class']].head(10)

,borrower_genders,male_borrowers,female_borrowers,borrowers_count,male_borrower_ratio,gender_class
0,female,0,1,1,0.0,female(s)
1,"female, female",0,2,2,0.0,female(s)
3,female,0,1,1,0.0,female(s)
4,female,0,1,1,0.0,female(s)
7,female,0,1,1,0.0,female(s)
8,female,0,1,1,0.0,female(s)
9,female,0,1,1,0.0,female(s)
10,female,0,1,1,0.0,female(s)
11,female,0,1,1,0.0,female(s)
12,female,0,1,1,0.0,female(s)


In [106]:
### Removing outliers based on loan_amount
# calculate z-scores for loan_amount (our outlier detection)
z_scores = zscore(data['loan_amount'])

# get boolean array indicating the presence of outliers
# using 2 & -2 z_scores to get 95% of data within 2 standard deviations
data['outlier_loan_amount'] = (z_scores > 2) | (z_scores < -2)


#removing outliers
data_clean = data[~data['outlier_loan_amount']]

# check amount of outliers
data['outlier_loan_amount'].sum()

19416

In [107]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 435525 entries, 0 to 221186
Data columns (total 21 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   435525 non-null  int64  
 1   funded_amount        435525 non-null  float64
 2   loan_amount          435525 non-null  float64
 3   activity             435525 non-null  object 
 4   sector               435525 non-null  object 
 5   country              435525 non-null  object 
 6   region               435525 non-null  object 
 7   posted_time          435525 non-null  object 
 8   disbursed_time       435525 non-null  object 
 9   funded_time          435525 non-null  object 
 10  term_in_months       435525 non-null  float64
 11  lender_count         435525 non-null  int64  
 12  borrower_genders     435525 non-null  object 
 13  repayment_interval   435525 non-null  object 
 14  date                 435525 non-null  object 
 15  male_borrowers       4

### Calculating funding duration (time between posted_time and funded_time)

In [108]:
loans = data_clean
# We would like to also include the time/duration between posted_time and funded_time - in other words: how long it takes to get a loan funded
# convert to pd.datetime
loans['posted_time'] = pd.to_datetime(loans['posted_time'])
loans['funded_time'] = pd.to_datetime(loans['funded_time'])

# calculate time between posted_time and funded_time
loans['funding_duration'] = loans['funded_time'] - loans['posted_time']

# the result in days instead
loans['funding_duration_days'] = (loans['funded_time'] - loans['posted_time']).dt.total_seconds() / (24 * 60 * 60)

# first rows to check
loans[['posted_time', 'funded_time', 'funding_duration', 'funding_duration_days']].head()

/var/folders/z7/j9rhznl118zbt_41ygz3rgyc0000gn/T/ipykernel_89346/3370879942.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loans['posted_time'] = pd.to_datetime(loans['posted_time'])
/var/folders/z7/j9rhznl118zbt_41ygz3rgyc0000gn/T/ipykernel_89346/3370879942.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loans['funded_time'] = pd.to_datetime(loans['funded_time'])
/var/folders/z7/j9rhznl118zbt_41ygz3rgyc0000gn/T/ipykernel_89346/3370879942.py:8: SettingWithCopyWarning: 
A value is trying to be set o

,posted_time,funded_time,funding_duration,funding_duration_days
0,2014-01-01 06:12:39+00:00,2014-01-02 10:06:32+00:00,1 days 03:53:53,1.162419
1,2014-01-01 06:51:08+00:00,2014-01-02 09:17:23+00:00,1 days 02:26:15,1.101562
3,2014-01-01 08:03:11+00:00,2014-01-01 13:00:00+00:00,0 days 04:56:49,0.206123
4,2014-01-01 11:53:19+00:00,2014-01-01 19:18:51+00:00,0 days 07:25:32,0.309398
7,2014-01-01 11:46:01+00:00,2014-01-10 18:18:44+00:00,9 days 06:32:43,9.272720


## SML Preperation

### SML Libaries importing

In [109]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder     # scaling numerical and encoding categorical data
from sklearn.compose import ColumnTransformer                       # applying transformations to specific columns
from sklearn.model_selection import RandomizedSearchCV              # hyperparameter tuning using random search
from sklearn.ensemble import VotingRegressor                        # combine multiple models using voting
import matplotlib.pyplot as plt                                     # plotting library for data visualization
import seaborn as sns                                               # advanced data visualization library
from sklearn.inspection import permutation_importance               # assessing feature importance through permutations
import matplotlib.pyplot as plt                                     # plotting library (imported twice, should remove one)

# SML models, cross-validation, and Regression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score   # performance metrics for regression models
from sklearn.model_selection import train_test_split                 # splitting data into training and testing sets
from sklearn.linear_model import LinearRegression                    # linear regression model
from sklearn.ensemble import RandomForestRegressor                   # Random forest regression model
from sklearn.model_selection import cross_val_score                  # Cross-validation scoring for models
from xgboost import XGBRegressor                                     # gradient boosting regressor from XGBoost
from imblearn.under_sampling import RandomUnderSampler                # undersampling technique for handling imbalanced datasets
from sklearn.tree import DecisionTreeRegressor                       # Decision tree regressor model

# Pipeline for models
from sklearn.pipeline import Pipeline                                


In [110]:
# Step 1: Identify numeric columns in the dataset
# We're selecting only the numeric columns for correlation analysis to check how related they are to each other.
numeric_columns = loans.select_dtypes(include=[np.number]).columns

# Step 2: Calculate the correlation matrix
# The correlation matrix tells us how strongly different numeric features are related (positively or negatively).
corr_matrix = loans[numeric_columns].corr()

# Step 3: Plot the correlation matrix
# We use a heatmap to visually display the correlations between features.
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
plt.title('Correlation Matrix for Numeric Columns in Loans Dataset')

# Step 4: Display the plot in a clean layout
# We ensure the plot fits the figure neatly before displaying.
plt.tight_layout()
plt.show()


### Filter relevant columns for SML

In [111]:
#Filtering only the relevant columns for the model
relevant_columns = [
    'loan_amount',
    'activity',
    'sector',
    'country',
    'region',
    'borrowers_count',
    'gender_class',
    'funding_duration_days',
    'term_in_months',
]


In [112]:
loans_df = loans[relevant_columns]

### Data Imbalance for country
Some countries account for more data than others, which could lead to imbalanced models.

In [113]:
# This line counts how many loans come from each country and displays the result.
# .value_counts() shows the frequency of loans per country, while .to_string() ensures the full list is printed without truncation.
print(loans_df['country'].value_counts().to_string())


In [114]:
# check initial distribution of countries
print("Country distribution:")
print(loans_df['country'].value_counts())

# Initialize RandomUnderSampler
rus = RandomUnderSampler(sampling_strategy='auto', random_state=9000)

# Apply resampling using 'country' as a balancing feature
X_resampled, _ = rus.fit_resample(loans_df, loans_df['country'])

print("Country distribution after balancing:")
print(X_resampled['country'].value_counts())

# updating dataframe
loans_df = X_resampled

In [115]:
# we know have substansially less data, which means it can be used directly for training models. 
loans_df.info()

In [116]:
# (OPTIONAL) Uncomment here to create a smaller sample for testing (5000 instead of 25360)
# loans_df = loans_df.sample(n=5000, random_state=9000)

### Setting up the target and feature variables + splitting data

In [117]:
# Set 'loan_amount' as the target variable we want to predict.
y = loans_df['loan_amount']

# Select features (columns) we will use to predict the loan amount.
X = loans_df[['activity', 'sector', 'country', 'borrowers_count', 'funding_duration_days', 'region', 'gender_class', 'term_in_months']]

# Prepare numeric features for scaling to ensure they're on the same scale.
numeric_features = ['borrowers_count', 'funding_duration_days', 'term_in_months']
numeric_transformer = StandardScaler()

# Prepare categorical features for encoding (turning them into numbers).
categorical_features = ['activity', 'sector', 'country', 'region', 'gender_class']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Combine both numeric and categorical preprocessing into one step.
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Split the data into training and testing sets (70% train, 30% test).
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=21)


### Creating pipelines

In [118]:
# Pipeline for Linear Regression
pipeline_lr = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Pipeline for Decision Tree Regression
pipeline_tree = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', DecisionTreeRegressor())
])

# Pipeline for Random Forest Regression
pipeline_rf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])
 
# Pipeline for XGBoost
pipeline_xgb = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor())
])

### Model Training without hyperparameterization

In [119]:
############################################################
# Train a Linear Regression model using the training data
pipeline_lr.fit(X_train, y_train)

# Use the trained model to predict the loan amount on the test data
y_pred_lr = pipeline_lr.predict(X_test)

# Evaluate the model by calculating MSE, MAE, and R² scores
mse_lr = mean_squared_error(y_test, y_pred_lr)
mae_lr = mean_absolute_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)

# Check the model's accuracy on the training and test data
train_score_lr = pipeline_lr.score(X_train, y_train)
test_score_lr = pipeline_lr.score(X_test, y_test)

# Print out the performance of the Linear Regression model
print("Linear Regression Performance:")
print(f"  MSE: {mse_lr:.4f}")
print(f"  MAE: {mae_lr:.4f}")
print(f"  R²: {r2_lr:.4f}")
print(f"Train Score: {train_score_lr:.4f}")
print(f"Test Score: {test_score_lr:.4f}")

############################################################
# Train a Random Forest model using the same process
pipeline_rf.fit(X_train, y_train)

# Predict the loan amount on the test data
y_pred_rf = pipeline_rf.predict(X_test)

# Evaluate the Random Forest model using the same metrics
mse_rf = mean_squared_error(y_test, y_pred_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

# Check accuracy on training and test sets
train_score_rf = pipeline_rf.score(X_train, y_train)
test_score_rf = pipeline_rf.score(X_test, y_test)

# Print out the Random Forest model's performance
print("\nRandom Forest Regression Performance:")
print(f"  MSE: {mse_rf:.4f}")
print(f"  MAE: {mae_rf:.4f}")
print(f"  R²: {r2_rf:.4f}")
print(f"Train Score: {train_score_rf:.4f}")
print(f"Test Score: {test_score_rf:.4f}")

############################################################
# Train an XGBoost model using the training data
pipeline_xgb.fit(X_train, y_train)

# Predict the loan amount on the test data
y_pred_rf = pipeline_xgb.predict(X_test)

# Evaluate XGBoost with the same performance metrics
mse_rf = mean_squared_error(y_test, y_pred_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

# Check the XGBoost model's accuracy on the training and test data
train_score_rf = pipeline_xgb.score(X_train, y_train)
test_score_rf = pipeline_xgb.score(X_test, y_test)

# Print out the XGBoost model's performance
print("\nXGBoost Performance:")
print(f"  MSE: {mse_rf:.4f}")
print(f"  MAE: {mae_rf:.4f}")
print(f"  R²: {r2_rf:.4f}")
print(f"Train Score: {train_score_rf:.4f}")
print(f"Test Score: {test_score_rf:.4f}")

############################################################


### Hyperparamter tuning using cross-validation

#### Random Forest Hyperparameter tuning

In [120]:
# Tuning Random Forest model to reduce overfitting by adjusting parameters
# Here we define a range of hyperparameters (e.g., n_estimators, max_depth) to try different settings for our Random Forest

param_distributions = {
    'regressor__n_estimators': [50, 100, 150],  # Number of trees in the forest
    'regressor__max_depth': [10, 15, 20],  # Maximum depth of each tree
    'regressor__min_samples_split': [5, 10, 15],  # Minimum samples required to split a node
    'regressor__min_samples_leaf': [3, 5, 7, 10]  # Minimum samples required to be at a leaf node
}

# We use RandomizedSearchCV to find the best combination of hyperparameters for our Random Forest model
random_search = RandomizedSearchCV(estimator=pipeline_rf, param_distributions=param_distributions,
                                   n_iter=15, cv=5, n_jobs=-1, scoring='r2', verbose=1, random_state=69)

# Train the model with different combinations of hyperparameters on the training data
random_search.fit(X_train, y_train)

# Print out the best set of hyperparameters and the corresponding cross-validation R² score
print(f"Best hyperparameters: {random_search.best_params_}")
print(f"Best cross-validation R^2 score: {random_search.best_score_}")

# Check how well the tuned model performs on both the training and test sets using R² score
train_r2 = random_search.score(X_train, y_train)
test_r2 = random_search.score(X_test, y_test)
print(f"Random Forest - Training R^2 score: {train_r2}")
print(f"Random Forest - Test R^2 score: {test_r2}")



In [121]:
# Pipeline for Random Forest Regression
pipeline_rf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=150, 
                                        min_samples_split=5, 
                                        min_samples_leaf=3,
                                        max_depth=20))
])

### Cross Validation (Random Forest + XGBoost)

In [122]:
# RANDOM FOREST MODEL

# Perform 5-fold cross-validation to assess model performance on different subsets of training data
cv_scores = cross_val_score(pipeline_rf, X_train, y_train, cv=5, scoring='r2')

# Train the model using the full training dataset
pipeline_rf.fit(X_train, y_train)

# Evaluate how well the model performs on the training and test datasets using R² score
train_score_rf = pipeline_rf.score(X_train, y_train)
test_score_rf = pipeline_rf.score(X_test, y_test)

# Predict the target variable on the test set
y_pred_rf = pipeline_rf.predict(X_test)

# Calculate the mean squared error (MSE) and root mean squared error (RMSE) to evaluate prediction accuracy
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)

# Output cross-validation scores, training/test R², and MSE/RMSE values
print(f"Random Forest")
print(f"Cross-Validation R^2 Scores: {cv_scores}")
print(f"Mean Cross-Validation R^2: {cv_scores.mean()}")
print(f'Random Forest - Training R^2 score: {train_score_rf}')
print(f'Random Forest - Test R^2 score: {test_score_rf}')
print(f'Random Forest - Test MSE: {mse_rf}')
print(f'Random Forest - Test RMSE: {rmse_rf}')

####################################################################################
# XGBOOST MODEL

# Perform 5-fold cross-validation to assess XGBoost model performance
cv_scores = cross_val_score(pipeline_xgb, X_train, y_train, cv=5, scoring='r2')

# Train the XGBoost model on the full training dataset
pipeline_xgb.fit(X_train, y_train)

# Evaluate the model on the training and test datasets using R² score
train_score_rf = pipeline_xgb.score(X_train, y_train)
test_score_rf = pipeline_xgb.score(X_test, y_test)

# Predict the target variable on the test set
y_pred_rf = pipeline_xgb.predict(X_test)

# Calculate MSE and RMSE for prediction accuracy
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)

# Output cross-validation scores, training/test R², and MSE/RMSE values
print(f"XGBoost")
print(f"Cross-Validation R^2 Scores: {cv_scores}")
print(f"Mean Cross-Validation R^2: {cv_scores.mean()}")
print(f'XGBoost - Training R^2 score: {train_score_rf}')
print(f'XGBoost - Test R^2 score: {test_score_rf}')
print(f'XGBoost - Test MSE: {mse_rf}')
print(f'XGBoost - Test RMSE: {rmse_rf}')


In [123]:

# ENSEMBLE MODEL - VOTING REGRESSOR
# We combine the Random Forest and XGBoost pipelines into one ensemble model.
# In this case, XGBoost has more weight (70%) as it tends to perform better for this dataset.

ensemble_model = VotingRegressor(
    estimators=[('rf', pipeline_rf), ('xgb', pipeline_xgb)],
    weights=[0.3, 0.7]  # 30% Random Forest, 70% XGBoost
)

# Fit the ensemble model on the training data
ensemble_model.fit(X_train, y_train)

# Evaluate the ensemble model on training data and print the R² score
ensemble_train_score = ensemble_model.score(X_train, y_train)
print(f"Ensemble Model Training R² Score: {ensemble_train_score:.4f}")

# Evaluate the ensemble model on test data to check its generalization
ensemble_test_score = ensemble_model.score(X_test, y_test)
print(f"Ensemble Model Test R² Score: {ensemble_test_score:.4f}")

# Comparing performance with the individual models (Random Forest & XGBoost)

# Random Forest: Train and evaluate
pipeline_rf.fit(X_train, y_train)
rf_train_score = pipeline_rf.score(X_train, y_train)
rf_test_score = pipeline_rf.score(X_test, y_test)
print(f"Random Forest Pipeline Training R² Score: {rf_train_score:.4f}")
print(f"Random Forest Pipeline Test R² Score: {rf_test_score:.4f}")

# XGBoost: Train and evaluate
pipeline_xgb.fit(X_train, y_train)
xgb_train_score = pipeline_xgb.score(X_train, y_train)
xgb_test_score = pipeline_xgb.score(X_test, y_test)
print(f"XGBoost Pipeline Training R² Score: {xgb_train_score:.4f}")
print(f"XGBoost Pipeline Test R² Score: {xgb_test_score:.4f}")

# Making predictions on the test set using the ensemble model
y_pred = ensemble_model.predict(X_test)


### Visulization of Ensemble Model

In [124]:
# Fit the ensemble model (VotingRegressor)
# The model is trained using the training data.
ensemble_model.fit(X_train, y_train)

# Calculate Permutation Feature Importance
# We use permutation importance to measure how each feature impacts the model's performance.
# The model's accuracy is checked with the feature randomly shuffled.
# n_repeats=10 means we shuffle the feature 10 times to get a more reliable estimate.
result = permutation_importance(ensemble_model, X_test, y_test, n_repeats=10, random_state=69, n_jobs=-1)

# Plotting the feature importance based on the results of permutation
# Features are sorted in ascending order for better visualization.
sorted_idx = result.importances_mean.argsort()

# Creating a horizontal bar plot to display the importance of each feature.
plt.figure(figsize=(10, 6))
plt.barh(range(len(sorted_idx)), result.importances_mean[sorted_idx], align="center")
plt.yticks(range(len(sorted_idx)), X_test.columns[sorted_idx])
plt.xlabel("Permutation Importance")
plt.title("Permutation Feature Importance for Voting Regressor")
plt.show()



In [125]:
# This line generates predictions for the test set based on the ensemble model we've built.
y_pred = ensemble_model.predict(X_test)

# Plot actual vs predicted values
# We create a scatter plot to compare the actual loan amounts (y_test) with the predicted loan amounts (y_pred).
# Adding transparency (alpha=0.3) and smaller markers (s=10) makes it easier to see dense data points.
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.3, s=10, color='blue', label='Predictions')  # marker size = (s=), transparency = (alpha=)

# Plotting the ideal fit line (red dashed line)
# This line represents a perfect prediction, where the actual and predicted loan amounts are identical.
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2, color='red', label='Ideal Fit')

# Adding plot title and axis labels for clarity
plt.title('Ensemble Model: Actual vs Predicted Loan Amounts', fontsize=16)
plt.xlabel('Actual Loan Amount', fontsize=14)
plt.ylabel('Predicted Loan Amount', fontsize=14)

# Add a legend to explain the blue dots (predictions) and the red line (ideal fit)
plt.legend()

# Adding a grid for better readability
plt.grid(True)

# Display the plot
plt.show()



## EXPORTING FILES FOR STREAMLIT 

### Exporting ensemble model to joblib format for usage in Streamlit


In [126]:
import joblib
joblib.dump(ensemble_model, 'ensemble_model.joblib')

['ensemble_model.joblib']

### Export target X_test (loan_amount) & y_test (features) to csv to be used in Streamlit deployment visualization 

In [127]:
X_test.to_csv('data/X_test.csv', index=False)
y_test.to_csv('data/y_test.csv', index=False)